# Calibration/Prediction with LSTM

Notice: we suppress training to only one epoch for conciseness of the script

In [15]:
# built-in pacakges
import torch.optim as optim
# in-house pacakages
from Utils import *
from Models import *

In [16]:
# GLOBAL VAR
DEVICE = torch.device("cuda")
TRAIN_FILE_NAME = "datasets/kaggle_train"
TEST_FILE_NAME = "datasets/kaggle_test"
BATCH_SIZE = 200
EMB_SIZE = 100
BEST_VALID_LOSS = float('inf')
device = torch.device("cuda")

In [17]:
# prepare data
train_iter, valid_iter, test_iter, text = tabularData(TRAIN_FILE_NAME, TEST_FILE_NAME, DEVICE, BATCH_SIZE, EMB_SIZE)
vocab = text.vocab

# Att'n Bi-Direction LSTM (Pre-Training)

In [18]:
# set up model/loss fun
VOCAB_SIZE = len(vocab)
WORD_DIM = EMB_SIZE; HID_DIM = 200; BIDIR = True
NUM_LAYERS = 2; NUM_CLASS = 3; ATT_DIM = 30; DROPOUT = 0.5
PAD_IDX = vocab.stoi[text.pad_token]; UNK_IDX = vocab.stoi[text.unk_token]
# initialize model
model = RNN(VOCAB_SIZE, WORD_DIM, HID_DIM, NUM_CLASS, NUM_LAYERS, BIDIR, DROPOUT, PAD_IDX, DEVICE, ATT_DIM).to(DEVICE)
# initialize word embedding
model.embedding.weight.data.copy_(vocab.vectors)
model.embedding.weight.data[UNK_IDX] = torch.zeros(WORD_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(WORD_DIM)
# initialize optimizer/loss functionals
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(DEVICE)

# train model
EPOCHS = 1
BEST_VALID_LOSS = float('inf')
for epoch in range(EPOCHS):
    train_loss, train_acc, train_att = training_LSTM(model, train_iter, optimizer, criterion, DEVICE)
    valid_loss, valid_acc = evaluation_LSTM(model, valid_iter, criterion)
    if valid_loss < BEST_VALID_LOSS:
        BEST_VALID_LOSS = valid_loss
#         torch.save(model.state_dict(), 'trained_model/text_lstm_model_attn.pt')

    print(f'Epoch: {epoch + 1:02}')
    print(f'\tTrain Loss(pre): {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

Epoch: 01
	Train Loss(pre): 0.944 | Train Acc: 53.51%
	 Val. Loss: 0.751 |  Val. Acc: 67.88%


# Bi-Direction LSTM + Att'n + Enhanced (Pre-Training)

In [19]:
# initialize model
model_plus = RNN(VOCAB_SIZE, WORD_DIM, HID_DIM, NUM_CLASS, NUM_LAYERS, BIDIR, DROPOUT, PAD_IDX, DEVICE, ATT_DIM, True).to(DEVICE)
# initialize word embedding
model_plus.embedding.weight.data.copy_(vocab.vectors)
model_plus.embedding.weight.data[UNK_IDX] = torch.zeros(WORD_DIM)
model_plus.embedding.weight.data[PAD_IDX] = torch.zeros(WORD_DIM)
# initialize optimizer/loss functionals
optimizer = optim.Adam(model_plus.parameters())
criterion = nn.CrossEntropyLoss().to(DEVICE)

# train model
EPOCHS = 1
BEST_VALID_LOSS = float('inf')
for epoch in range(EPOCHS):
    train_loss, train_acc, train_att = training_LSTM(model_plus, train_iter, optimizer, criterion, DEVICE)
    valid_loss, valid_acc = evaluation_LSTM(model_plus, valid_iter, criterion)
    if valid_loss < BEST_VALID_LOSS:
        BEST_VALID_LOSS = valid_loss
        word_emb = saveWordVecFromModel(vocab.itos, model_plus.embedding.weight.data)
#         torch.save(model_plus.state_dict(), 'trained_model/text_lstm_model_attn_plus.pt')
 
    print(f'Epoch: {epoch + 1:02}')
    print(f'\tTrain Loss(pre): {train_loss:.3f} | Train Loss(att): {train_att:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

Epoch: 01
	Train Loss(pre): 0.953 | Train Loss(att): 1.665 | Train Acc: 52.09%
	 Val. Loss: 0.823 |  Val. Acc: 60.16%


# Bi-Direction LSTM + Att'n + Enhanced (Fine-Tuning)

In [20]:
# Switch Dataset
TRAIN_FILE_NAME = "datasets/covid_train"
TEST_FILE_NAME = "datasets/covid_test"
# prepare data
train_iter, valid_iter, test_iter, text_covid = tabularData(TRAIN_FILE_NAME, TEST_FILE_NAME, DEVICE, BATCH_SIZE, EMB_SIZE, 0.9, 1, False)
vocab_covid = text_covid.vocab
# load pre-trained word_embeddings
updated_wv = updateWordVecToModel(vocab_covid.itos, vocab_covid.vectors, word_emb, DEVICE)

In [7]:
# construct model
VOCAB_SIZE = len(vocab_covid)
WORD_DIM = EMB_SIZE; HID_DIM = 200; BIDIR = True
NUM_LAYERS = 2; NUM_CLASS = 3; ATT_DIM = 30; DROPOUT = 0.5
PAD_IDX = vocab_covid.stoi[text_covid.pad_token]; UNK_IDX = vocab_covid.stoi[text_covid.unk_token]
# initialize model
model_covid = RNN(VOCAB_SIZE, WORD_DIM, HID_DIM, NUM_CLASS, NUM_LAYERS, BIDIR, DROPOUT, PAD_IDX, DEVICE, ATT_DIM).to(DEVICE)
model_covid.embedding.weight.data.copy_(updated_wv)
model_covid.embedding.weight.data[UNK_IDX] = torch.zeros(WORD_DIM)
model_covid.embedding.weight.data[PAD_IDX] = torch.zeros(WORD_DIM)
# initialize optimizer/loss functionals
optimizer = optim.Adam(model_covid.parameters())
criterion = nn.CrossEntropyLoss().to(DEVICE)

In [8]:
# train model
EPOCHS = 1
BEST_VALID_LOSS = float('inf')
for epoch in range(EPOCHS):
    train_loss, train_acc, train_att = training_LSTM(model_covid, train_iter, optimizer, criterion, DEVICE)
    valid_loss, valid_acc = evaluation_LSTM(model_covid, valid_iter, criterion)
    if valid_loss < BEST_VALID_LOSS:
        BEST_VALID_LOSS = valid_loss
#         torch.save(model_covid.state_dict(), 'trained_model/lstm_covid19.pt')

    print(f'Epoch: {epoch + 1:02}')
    print(f'\tTrain Loss(pre): {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

Epoch: 01
	Train Loss(pre): 0.917 | Train Acc: 56.56%
	 Val. Loss: 0.696 |  Val. Acc: 73.28%


# Results Analyzer

In [9]:
model_1 = RNN(VOCAB_SIZE, WORD_DIM, HID_DIM, NUM_CLASS, NUM_LAYERS, BIDIR, DROPOUT, PAD_IDX, DEVICE, ATT_DIM).to(DEVICE)
model_1.load_state_dict(torch.load("trained_model/lstm_covid19.pt"), strict=False)

<All keys matched successfully>

In [10]:
# Check example
inx = 151
sen = vars(test_iter.dataset.examples[inx])['text']
res, attention = extract_att_weights(model_1, sen, vocab_covid, DEVICE)
show_attention(sen, attention)

,0,1,2,3,4,5,6,7
0,i,pray,my,haters,die,of,corona,virus
